# Basic Imports

In [68]:
import os
import requests
!pip install meteostat
import meteostat
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Mounting GDrive

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
dirs = [d for d in os.listdir('/content/drive/MyDrive/') if os.path.isdir(os.path.join('/content/drive/MyDrive/', d))]
dirs

['Colab Notebooks',
 'MAD 1 Project Demo',
 'MAD 2 Project Demo',
 'TDS_PROJECT_1',
 'BDM_CAPSTONE_PROJECT']

In [29]:
if dirs[-1] == 'BDM_CAPSTONE_PROJECT':
  !ls /content/drive/MyDrive/{dirs[-1]}

CHARTS	COLAB_NOTEBOOK	DATA  PROOF_OF_ORIGINALITY  SUPPLEMENTS


In [30]:
%cd /content/drive/MyDrive/{dirs[-1]}/DATA

/content/drive/MyDrive/BDM_CAPSTONE_PROJECT/DATA


In [31]:
df = pd.read_csv('data.csv')
df.head()

,DATE,DEMAND,FROM,TO,DISTANCE,PROFIT,LAGAN
0,2024-07-22,1.0,Masaurhi,NaN,132.0,1000.0,0.0
1,2024-07-21,1.0,Masaurhi,NaN,46.0,1000.0,0.0
2,2024-07-20,0.0,NaN,NaN,NaN,NaN,0.0
3,2024-07-19,0.0,NaN,NaN,NaN,NaN,0.0
4,2024-07-18,0.0,NaN,NaN,NaN,NaN,0.0


# Sourcing Location Data

In [32]:
unique_to = df['TO'].unique()
unique_from = df['FROM'].unique()
all_addresses = set(unique_to).union(set(unique_from))
all_addresses = {loc for loc in all_addresses if pd.notna(loc)}
all_addresses

{'Akbarpur Ranipur',
 'Arrah',
 'Athmal Gola',
 'Barni',
 'Bhagwanganj',
 'Bharatpura',
 'Bihar Sharif',
 'Bihta',
 'Bishunpur',
 'Dahiya',
 'Daulatpur',
 'Dhanuki',
 'Diara Malahi',
 'Fatehpur',
 'Fatwah',
 'Gaya',
 'Hazipur',
 'Hilsa',
 'Hulasganj',
 'Imaliya',
 'Imamganj',
 'Jehanabad',
 'Kadirganj',
 'Koilwar',
 'Kurtha',
 'Kurthoul',
 'Makhdumpur',
 'Masaurhi',
 'Muzaffarpur',
 'Nadaul',
 'Nadauna',
 'Nadwan',
 'Nawada',
 'Okri',
 'Parawalpur',
 'Patna',
 'Piyar pura',
 'Punpun',
 'Rajgir',
 'Sarwan',
 'Siwan'}

In [33]:
geolocator = Nominatim(user_agent="sugamsawaarigeocoder")
location_dict = {}
for address in all_addresses :
    location = geolocator.geocode(address + ', ' + 'Bihar' + ', ' + 'India')
    if location:
        location_dict[address] = (location.latitude, location.longitude)
location_dict

{'Punpun': (25.4639089, 85.11218926787205),
 'Fatehpur': (24.60839765, 85.22643708983149),
 'Hilsa': (25.30068345, 85.24938291069174),
 'Hazipur': (24.9490651, 85.6148287),
 'Koilwar': (25.5801196, 84.75725647778808),
 'Athmal Gola': (25.4534509, 85.6219947),
 'Bishunpur': (25.0869013, 87.2005144),
 'Dhanuki': (25.1496832, 85.6053004),
 'Nadwan': (25.3982109, 85.0687914),
 'Daulatpur': (25.1806579, 86.7681299),
 'Patna': (25.6093239, 85.1235252),
 'Imamganj': (24.406483350000002, 84.629212130725),
 'Imaliya': (25.2844638, 85.0946715),
 'Kurtha': (25.13326045, 84.81126025770061),
 'Arrah': (25.62345725, 84.59683868653386),
 'Nawada': (24.817324900000003, 85.51790846584018),
 'Makhdumpur': (25.073940399999998, 84.98787293898746),
 'Masaurhi': (25.3516471, 84.98734798892505),
 'Kurthoul': (25.5601145, 85.1343521),
 'Bihta': (25.56181525, 84.86694757191393),
 'Hulasganj': (25.0878322, 85.14966925645304),
 'Nadaul': (25.2965388, 85.0173352),
 'Jehanabad': (25.15295785, 85.00650426458421),
 

In [36]:
location_df = pd.DataFrame.from_dict(location_dict, orient='index', columns=['LATITUDE', 'LONGITUDE'])
location_df.index.name = 'LOCATION'
location_df.reset_index(inplace=True)
location_df

,LOCATION,LATITUDE,LONGITUDE
0,Punpun,25.463909,85.112189
1,Fatehpur,24.608398,85.226437
2,Hilsa,25.300683,85.249383
3,Hazipur,24.949065,85.614829
4,Koilwar,25.580120,84.757256
5,Athmal Gola,25.453451,85.621995
6,Bishunpur,25.086901,87.200514
7,Dhanuki,25.149683,85.605300
8,Nadwan,25.398211,85.068791
9,Daulatpur,25.180658,86.768130


In [43]:
location_df.to_csv('location.csv', index=False)
!ls

data.csv  fuel.csv  location.csv


# Sourcing Weather Data

In [45]:
min_date = df['DATE'].min()
max_date = df['DATE'].max()
print("Minimum Date:", min_date)
print("Maximum Date:", max_date)

Minimum Date: 2022-08-01
Maximum Date: 2024-07-22


In [46]:
location_df[location_df['LOCATION'] == 'Patna']

,LOCATION,LATITUDE,LONGITUDE
10,Patna,25.609324,85.123525


In [69]:
start = datetime(2022, 8, 1)
end = datetime(2024, 7, 22)
location = meteostat.Point(25.61, 85.12)
data = meteostat.Daily(location, start, end)
data = data.fetch()

In [77]:
weather_df = data[['tmin', 'tmax', 'prcp']].reset_index()
weather_df

,time,tmin,tmax,prcp
0,2022-08-01,25.8,33.6,0.5
1,2022-08-02,26.0,33.0,7.1
2,2022-08-03,26.0,33.2,37.1
3,2022-08-04,26.5,33.0,0.5
4,2022-08-05,27.7,35.5,0.0
...,...,...,...,...
717,2024-07-18,32.0,37.0,0.1
718,2024-07-19,32.0,37.0,2.4
719,2024-07-20,32.0,36.0,0.1
720,2024-07-21,31.0,35.0,0.4


In [78]:
weather_df = weather_df.rename(columns={'time': 'DATE', 'tmin': 'TMIN', 'tmax': 'TMAX', 'prcp': 'PRCP'})
weather_df

,DATE,TMIN,TMAX,PRCP
0,2022-08-01,25.8,33.6,0.5
1,2022-08-02,26.0,33.0,7.1
2,2022-08-03,26.0,33.2,37.1
3,2022-08-04,26.5,33.0,0.5
4,2022-08-05,27.7,35.5,0.0
...,...,...,...,...
717,2024-07-18,32.0,37.0,0.1
718,2024-07-19,32.0,37.0,2.4
719,2024-07-20,32.0,36.0,0.1
720,2024-07-21,31.0,35.0,0.4


In [79]:
weather_df.to_csv('weather.csv', index=False)
!ls

data.csv  fuel.csv  location.csv  weather.csv
